<a href="https://colab.research.google.com/github/KyoungaRyu/rnn-pytorch/blob/master/Naver_Sentiment_Movie_Corpus(Pytorch2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** konlpy와 한국어 형태소 분석기(okt) 설치(Importing a library that is not in Colaboratory)**
https://colab.research.google.com/notebooks/snippets/importing_libraries.ipynb


In [0]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy twitter

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages

#[Preparing Data]

**한국어형태소분석기 성능비교 확인 후 Open Korean Text로 결정**
   https://iostream.tistory.com/144
   
Okt(twitter) import 방법 딥러닝을 이용한 자연어 처리 입문 https://wikidocs.net/21698

In [0]:
import konlpy
from konlpy.tag import Okt
twitter = Okt()

/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


**Code Reference- Field 선언**

(https://zzsza.github.io/data/2018/03/17/pytorch-rnn/)

In [0]:
import torch
from torchtext import data
from torchtext.data import TabularDataset

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = twitter.morphs, include_lengths = True) # text 길이 옵션 넣어줘야 함
LABEL = data.LabelField(dtype = torch.float)



** Dataset 로드**

검색: pytorch tabular dataset
https://torchtext.readthedocs.io/en/latest/data.html#tabulardataset

In [0]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

train_data = TabularDataset(path="/content/gdrive/My Drive/Colab Notebooks/data/ratings.txt", # 데이터가 있는 root 경로 
                            format='tsv', # \t로 구분
                            skip_header=True, # 헤더가 있다면 스킵
                            fields=[('id', None), ('text', TEXT), ('label', LABEL)])


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))
train_data, test_data = train_data.split(random_state = random.seed(SEED))


**  TEXT와 LABEL 필드로부터 Vocabulary 생성**

In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [00:15, 55.4MB/s]                           
100%|█████████▉| 398859/400000 [00:19<00:00, 20066.00it/s]

In [0]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [0]:
print(vars(train_data[0]))

{'text': ['재밋네', '요', 'ㅋㅋㅋ', '생각', '보다', '재밋게봄'], 'label': '1'}


7. torchtext.data.Bucketiterator로부터 데이터 Iterator 생성

-  pytorch bucketiterator example 검색
1) https://torchtext.readthedocs.io/en/latest/examples.html
2) https://github.com/pytorch/text


8. sort_key=lambda 검색
= Sort a data list by caption length 
https://www.programcreek.com/python/example/100895/torch.utils.data.sort

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),      #def sort(data):/ return len(data.text)라는 형식으로 이해함
    sort_within_batch=True,           
    device = device)

#[Build the Model]

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                bidirectional, dropout, pad_idx):
      super().__init__()
      self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
      self.rnn = nn.LSTM(embedding_dim, 
                         hidden_dim, 
                         num_layers=n_layers, 
                         bidirectional=bidirectional, 
                         dropout=dropout)
      self.fc = nn.Linear(hidden_dim * 2, output_dim)
      self.dropout = nn.Dropout(dropout)
      
    def forward(self, text,  text_lengths):
      embedded = self.dropout(self.embedding(text))
    
      # 길이가 0인 데이터가 있는지 확인
      zeroText, _ = (text_lengths == 0).nonzero().size()
      
      # 길이가 0인 데이터가 없으면 packing
      if zeroText == 0:
        embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
      
      output, (hidden, cell) = self.rnn(embedded)
       
      # 길이가 0인 데이터가 없으면 packing 풀기
      if zeroText == 0:
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(output)

      hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
      
      return self.fc(hidden)

In [0]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


#[Train the Model]

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [0]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

정확도 계산 함수

In [0]:
def binary_accuracy(preds, y):
    
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y). float()
    acc = correct.sum() / len(correct)
    return acc

Training 함수

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

평가 함수

In [0]:
def evaluate(model, iterator, criterion):
  
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
       for batch in iterator:
          text, text_lengths = batch.text
          predictions = model(text, text_lengths).squeeze(1)
          loss = criterion(predictions, batch.label)
          acc = binary_accuracy(predictions, batch.label)
          
          epoch_loss += loss.item()
          epoch_acc += acc.item()
          
    return epoch_loss/ len(iterator), epoch_acc / len(iterator)

학습 시간 출력 함수

In [0]:
import time

def epoch_time(start_time, end_time): 
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
    

Training

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

100%|█████████▉| 398859/400000 [00:30<00:00, 20066.00it/s]

Epoch: 01 | Epoch Time: 0m 43s
	Train Loss: 0.544 | Train Acc: 71.59%
	 Val. Loss: 0.415 |  Val. Acc: 80.59%
Epoch: 02 | Epoch Time: 0m 42s
	Train Loss: 0.432 | Train Acc: 79.66%
	 Val. Loss: 0.370 |  Val. Acc: 83.03%
Epoch: 03 | Epoch Time: 0m 42s
	Train Loss: 0.383 | Train Acc: 82.44%
	 Val. Loss: 0.359 |  Val. Acc: 84.02%
Epoch: 04 | Epoch Time: 0m 42s
	Train Loss: 0.354 | Train Acc: 84.10%
	 Val. Loss: 0.341 |  Val. Acc: 85.14%
Epoch: 05 | Epoch Time: 0m 42s
	Train Loss: 0.331 | Train Acc: 85.42%
	 Val. Loss: 0.329 |  Val. Acc: 85.49%
